In [1]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Lambda
from keras.layers import Activation
from matplotlib import pyplot
from keras import backend

Using TensorFlow backend.


In [10]:
# custom activation function 
def custom_activation(output):
    logexpsum = backend.sum(backend.exp(output), axis=-1, keepdims=True)
    result = logexpsum / (logexpsum + 1.0)
    return result
# define supervised and unsupervised distriminator models
def define_discriminator(in_shape=(28,28,1), n_classes=10):
    #image input
    in_image = Input(shape=in_shape)
    #downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(in_image)
    fe = LeakyReLU(alpha=0.2)(fe)
    #downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    #downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)

    #flatten feature maps
    fe = Flatten()(fe)
    #dropout
    fe = Dropout(0.4)(fe)
    #output layer nodes
    fe = Dense(n_classes)(fe)
    #supervised output
    c_out_layer = Activation('softmax')(fe)
    # define and complie the supervised
    c_model = Model(in_image, c_out_layer)
    c_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
    
    #unsupervised output
    d_out_layer = Lambda(custom_activation)(fe)
    #define and compile unsupervised
    d_model = Model(in_image, d_out_layer)
    d_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
    return d_model, c_model

# define generator
def define_generator(latent_dim):
    #image generator input
    in_lat = Input(shape=(latent_dim,))
    n_nodes = 128 * 7 * 7
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((7, 7, 128))(gen)
    #upsample to 14 x14 
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    #upsample to 28 x 28
    gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # output 
    out_layer = Conv2D(1, (7, 7), activation='tanh', padding='same')(gen)
    #define model
    model = Model(in_lat, out_layer)
    return model

# define the commbined model for genreator and discriminator model
def define_gan(g_model, d_model):
    # make weights not trainable 
    d_model.trainable = False
    #connect image output from generator as input to discriminator 
    gan_output = d_model(g_model.output)
    model = Model(g_model.input, gan_output)
    # compile model 
    opt = Adam(lr=0.002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

def load_real_samples():
    # load dataset
    (trainX, trainy), (_, _) = load_data()
    # expand to 3d, e.g. add channels
    X = expand_dims(trainX, axis=-1)
    # convert from ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    print(X.shape, trainy.shape)
    return [X, trainy]
# select a supervised subset of the dataset, ensures classes are balanced
def select_supervised_samples(dataset, n_samples=100, n_classes=10):
    X, y = dataset
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes)
    for i in range(n_classes):
        # get all images for this class
        X_with_class = X[y == i]
        # choose random instances
        ix = randint(0, len(X_with_class), n_per_class)
        # add to list
        [X_list.append(X_with_class[j]) for j in ix]
        [y_list.append(i) for j in ix]
    return asarray(X_list), asarray(y_list)

# select real samples
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return [X, labels], y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    z_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = z_input.reshape(n_samples, latent_dim)
    return z_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict(z_input)
    # create class labels
    y = zeros((n_samples, 1))
    return images, y

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, c_model, latent_dim, dataset, n_samples=100):
    # prepare fake examples
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    # plot images
    for i in range(100):
        # define subplot
        pyplot.subplot(10, 10, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
    # save plot to file
    filename1 = 'generated_plot_%04d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # evaluate the classifier model
    X, y = dataset
    _, acc = c_model.evaluate(X, y, verbose=0)
    print('Classifier Accuracy: %.3f%%' % (acc * 100))
    # save the generator model
    filename2 = 'g_model_%04d.h5' % (step+1)
    g_model.save(filename2)
    # save the classifier model
    filename3 = 'c_model_%04d.h5' % (step+1)
    c_model.save(filename3)
    print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))
    
  # train the generator and discriminator
def train(g_model, d_model, c_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=100):
    # select supervised dataset
    X_sup, y_sup = select_supervised_samples(dataset)
    print(X_sup.shape, y_sup.shape)
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, n_batch, half_batch, bat_per_epo, n_steps))
    # manually enumerate epochs
    for i in range(n_steps):
        # update supervised discriminator (c)
        [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], half_batch)
        c_loss, c_acc = c_model.train_on_batch(Xsup_real, ysup_real)
        # update unsupervised discriminator (d)
        [X_real, _], y_real = generate_real_samples(dataset, half_batch)
        d_loss1 = d_model.train_on_batch(X_real, y_real)
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        d_loss2 = d_model.train_on_batch(X_fake, y_fake)
        # update generator (g)
        X_gan, y_gan = generate_latent_points(latent_dim, n_batch), ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
        # summarize loss on this batch
        print('>%d, c[%.3f,%.0f], d[%.3f,%.3f], g[%.3f]' % (i+1, c_loss, c_acc*100, d_loss1, d_loss2, g_loss))
        # evaluate the model performance every so often
        if (i+1) % (bat_per_epo * 1) == 0:
            summarize_performance(i, g_model, c_model, latent_dim, dataset)  

In [2]:
# example of custom activation function
import numpy as np

# custom activation function
def custom_activation(output):
    logexpsum = np.sum(np.exp(output))
    result = logexpsum / (logexpsum + 1.0)
    return result

# all -10s
output = np.asarray([-10.0, -10.0, -10.0])
print(custom_activation(output))
# all -1s
output = np.asarray([-1.0, -1.0, -1.0])
print(custom_activation(output))
# all 0s
output = np.asarray([0.0, 0.0, 0.0])
print(custom_activation(output))
# all 1s
output = np.asarray([1.0, 1.0, 1.0])
print(custom_activation(output))
# all 10s
output = np.asarray([10.0, 10.0, 10.0])
print(custom_activation(output))

0.00013618124143106674
0.5246331135813284
0.75
0.890768227426964
0.9999848669190928


In [11]:
# size of the latent space
latent_dim = 100
# create the discriminator models
d_model, c_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, c_model, gan_model, dataset, latent_dim)

11493376/11490434 [==============================] - 4s 0us/step
(60000, 28, 28, 1) (60000,)
(100, 28, 28, 1) (100,)
n_epochs=20, n_batch=100, 1/2=50, b/e=600, steps=12000
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, c[2.313,8], d[0.093,2.399], g[0.095]
>2, c[2.306,6], d[0.088,2.428], g[0.094]
>3, c[2.307,20], d[0.087,2.458], g[0.094]
>4, c[2.277,12], d[0.090,2.404], g[0.100]
>5, c[2.280,20], d[0.095,2.333], g[0.109]
>6, c[2.286,18], d[0.100,2.246], g[0.122]
>7, c[2.250,28], d[0.108,2.122], g[0.144]
>8, c[2.298,10], d[0.120,1.967], g[0.183]
>9, c[2.246,22], d[0.139,1.723], g[0.249]
>10, c[2.264,18], d[0.171,1.451], g[0.364]
>11, c[2.211,22], d[0.222,1.162], g[0.536]
>12, c[2.196,20], d[0.320,2.036], g[0.156]
>13, c[2.255,20], d[0.318,3.061], g[0.061]
>14, c[2.233,24], d[0.265,3.017], g[0.065]
>15, c[2.127,32], d[0.218,2.786], g[0.078]
>16, c[2.091,42], d[0.178,2.519], g[0.096]
>17, c[2.103,40], d[0.160,2.339], g[0.115]
>18, c[2.045,52], d[0.136,2.155], g[0.146]
>19, c[2.072,26], d[0.120,1.905], g[0.202]
>20, c[2.010,48], d[0.102,1.567], g[0.326]
>21, c[1.961,32], d[0.086,1.127], g[0.609]
>22, c[1.937,52], d[0.069,0.737], g[0.986]
>23, c[1.823,56], d[0.055,1.058], g[0.713]
>24, c[1.914,42], d[0.

>189, c[0.129,98], d[0.712,0.958], g[0.802]
>190, c[0.121,100], d[0.845,0.960], g[0.891]
>191, c[0.166,100], d[0.881,1.107], g[0.820]
>192, c[0.109,98], d[1.042,1.019], g[0.897]
>193, c[0.127,100], d[0.883,0.944], g[0.757]
>194, c[0.097,100], d[0.948,0.970], g[0.840]
>195, c[0.139,98], d[0.788,1.035], g[0.965]
>196, c[0.175,98], d[0.992,0.842], g[0.916]
>197, c[0.152,98], d[0.804,0.985], g[0.810]
>198, c[0.133,100], d[0.861,0.848], g[0.861]
>199, c[0.123,98], d[0.784,0.908], g[0.806]
>200, c[0.087,100], d[0.757,1.159], g[0.856]
>201, c[0.089,100], d[0.838,1.145], g[0.909]
>202, c[0.121,100], d[0.877,0.917], g[0.963]
>203, c[0.132,98], d[0.938,0.811], g[0.852]
>204, c[0.099,100], d[0.921,0.959], g[0.834]
>205, c[0.125,100], d[1.083,1.062], g[0.753]
>206, c[0.189,94], d[0.938,1.196], g[0.804]
>207, c[0.136,100], d[0.887,1.027], g[0.721]
>208, c[0.181,96], d[0.825,0.964], g[0.818]
>209, c[0.110,100], d[0.715,0.943], g[0.831]
>210, c[0.134,100], d[0.720,0.971], g[0.828]
>211, c[0.096,100],

>372, c[0.055,100], d[0.976,0.973], g[0.896]
>373, c[0.074,100], d[0.819,0.943], g[0.927]
>374, c[0.054,100], d[0.967,0.892], g[0.944]
>375, c[0.064,100], d[0.827,0.861], g[0.923]
>376, c[0.067,100], d[0.776,0.947], g[0.946]
>377, c[0.053,100], d[0.979,0.841], g[0.836]
>378, c[0.097,98], d[0.975,0.971], g[0.897]
>379, c[0.074,100], d[0.854,0.968], g[0.933]
>380, c[0.061,100], d[0.737,0.977], g[0.951]
>381, c[0.080,100], d[0.930,0.860], g[1.039]
>382, c[0.076,100], d[0.758,0.777], g[0.844]
>383, c[0.051,100], d[0.990,0.982], g[0.914]
>384, c[0.093,100], d[0.621,0.911], g[0.934]
>385, c[0.065,100], d[0.707,0.760], g[1.061]
>386, c[0.069,100], d[0.770,0.796], g[0.940]
>387, c[0.051,100], d[0.875,1.184], g[0.955]
>388, c[0.062,100], d[0.830,0.947], g[0.990]
>389, c[0.085,100], d[0.862,0.793], g[0.984]
>390, c[0.070,100], d[0.627,0.845], g[1.004]
>391, c[0.049,100], d[0.794,0.970], g[1.006]
>392, c[0.060,100], d[0.906,1.067], g[0.942]
>393, c[0.063,100], d[0.953,1.056], g[1.060]
>394, c[0.0

...

>9860, c[0.000,100], d[0.000,0.000], g[15.896]
>9861, c[0.000,100], d[0.000,0.000], g[16.073]
>9862, c[0.000,100], d[0.001,0.000], g[16.118]
>9863, c[0.000,100], d[0.000,0.000], g[16.118]
>9864, c[0.000,100], d[0.000,0.000], g[16.115]
>9865, c[0.000,100], d[0.000,0.000], g[16.064]
>9866, c[0.000,100], d[0.000,0.000], g[16.046]
>9867, c[0.000,100], d[0.000,0.000], g[16.118]
>9868, c[0.000,100], d[0.000,0.000], g[16.118]
>9869, c[0.000,100], d[0.000,0.000], g[16.017]
>9870, c[0.000,100], d[0.000,0.000], g[16.093]
>9871, c[0.000,100], d[0.000,0.000], g[16.048]
>9872, c[0.000,100], d[0.000,0.000], g[16.102]
>9873, c[0.000,100], d[0.000,0.000], g[16.053]
>9874, c[0.000,100], d[0.000,0.000], g[16.040]
>9875, c[0.000,100], d[0.000,0.000], g[16.032]
>9876, c[0.000,100], d[0.000,0.000], g[16.110]
>9877, c[0.000,100], d[0.000,0.000], g[16.118]
>9878, c[0.000,100], d[0.000,0.000], g[16.118]
>9879, c[0.000,100], d[0.000,0.000], g[16.111]
>9880, c[0.000,100], d[0.000,0.000], g[16.064]
>9881, c[0.00

>10034, c[0.000,100], d[0.000,0.000], g[15.747]
>10035, c[0.000,100], d[0.000,0.000], g[15.568]
>10036, c[0.000,100], d[0.000,0.000], g[15.877]
>10037, c[0.000,100], d[0.000,0.000], g[15.852]
>10038, c[0.000,100], d[0.000,0.000], g[15.788]
>10039, c[0.000,100], d[0.000,0.000], g[15.660]
>10040, c[0.000,100], d[0.000,0.000], g[15.560]
>10041, c[0.000,100], d[0.000,0.000], g[15.829]
>10042, c[0.000,100], d[0.000,0.000], g[15.629]
>10043, c[0.000,100], d[0.000,0.000], g[15.788]
>10044, c[0.000,100], d[0.000,0.000], g[15.707]
>10045, c[0.000,100], d[0.000,0.000], g[15.685]
>10046, c[0.000,100], d[0.000,0.000], g[15.715]
>10047, c[0.000,100], d[0.000,0.143], g[16.118]
>10048, c[0.000,100], d[0.000,0.000], g[16.118]
>10049, c[0.000,100], d[0.049,0.000], g[15.930]
>10050, c[0.000,100], d[0.000,0.000], g[15.781]
>10051, c[0.000,100], d[0.000,0.000], g[15.742]
>10052, c[0.000,100], d[0.000,0.000], g[15.628]
>10053, c[0.000,100], d[0.000,0.000], g[15.796]
>10054, c[0.000,100], d[0.000,0.000], g[

>10203, c[0.000,100], d[0.000,0.000], g[16.118]
>10204, c[0.000,100], d[0.000,0.000], g[16.094]
>10205, c[0.000,100], d[0.000,0.000], g[16.118]
>10206, c[0.000,100], d[0.000,0.000], g[16.118]
>10207, c[0.000,100], d[0.000,0.000], g[16.118]
>10208, c[0.000,100], d[0.000,0.000], g[16.118]
>10209, c[0.000,100], d[0.000,0.000], g[16.118]
>10210, c[0.000,100], d[0.000,0.000], g[16.118]
>10211, c[0.000,100], d[0.001,0.000], g[16.088]
>10212, c[0.000,100], d[0.000,0.000], g[16.118]
>10213, c[0.000,100], d[0.000,0.000], g[16.118]
>10214, c[0.000,100], d[0.000,0.000], g[16.118]
>10215, c[0.000,100], d[0.000,0.000], g[16.083]
>10216, c[0.000,100], d[0.000,0.000], g[16.118]
>10217, c[0.000,100], d[0.000,0.000], g[16.118]
>10218, c[0.000,100], d[0.000,0.000], g[16.118]
>10219, c[0.000,100], d[0.008,0.000], g[16.096]
>10220, c[0.000,100], d[0.000,0.000], g[16.118]
>10221, c[0.000,100], d[0.000,0.000], g[15.994]
>10222, c[0.000,100], d[0.000,0.000], g[16.104]
>10223, c[0.000,100], d[0.000,0.000], g[

>10374, c[0.000,100], d[0.000,0.000], g[16.118]
>10375, c[0.000,100], d[0.000,0.000], g[16.118]
>10376, c[0.000,100], d[0.000,0.000], g[16.118]
>10377, c[0.000,100], d[0.000,0.000], g[16.118]
>10378, c[0.000,100], d[0.000,0.000], g[16.118]
>10379, c[0.000,100], d[0.000,0.000], g[16.118]
>10380, c[0.000,100], d[0.000,0.000], g[16.113]
>10381, c[0.000,100], d[0.000,0.000], g[16.067]
>10382, c[0.000,100], d[0.000,0.000], g[16.077]
>10383, c[0.000,100], d[0.000,0.000], g[16.112]
>10384, c[0.000,100], d[0.000,0.000], g[16.118]
>10385, c[0.000,100], d[0.000,0.000], g[16.118]
>10386, c[0.000,100], d[0.000,0.000], g[16.118]
>10387, c[0.000,100], d[0.000,0.000], g[16.048]
>10388, c[0.000,100], d[0.000,0.000], g[16.056]
>10389, c[0.000,100], d[0.000,0.000], g[16.114]
>10390, c[0.000,100], d[0.000,0.000], g[16.118]
>10391, c[0.000,100], d[0.000,0.000], g[16.118]
>10392, c[0.000,100], d[0.000,0.000], g[16.118]
>10393, c[0.000,100], d[0.000,0.000], g[16.118]
>10394, c[0.000,100], d[0.000,0.000], g[

KeyboardInterrupt: 